In [3]:
# General setup, as explained earlier
import os
from pprint import pprint
from urllib3.util.retry import Retry

import requests
from requests.adapters import HTTPAdapter

PLANET_API_KEY = os.getenv('PL_API_KEY')  # Or set it explicitly

session = requests.Session()
session.auth = (PLANET_API_KEY, '')

retries = Retry(total=5,
                backoff_factor=0.1,  
                status_forcelist=[429])

session.mount('https://', HTTPAdapter(max_retries=retries))

Search Filters and Stats Endpoint
--------------------------------------------------

There's a lot of data available on the Planet Data API and in order to find what we're looking for, we'll have to perform various types of searches. To construct a search, we use **filters** that let us limit results based on date, geography or other metadata properties.

Using these filters, we are able to search for items or get statistics for search results. Understanding how filters work is key to using Planet's api effectively.  In this exercise, we'll experiment with using filters on the **stats** endpoint to the api before continuing on to searching for scenes.

The `/stats` endpoint provides a summary of the available data based on our search criteria.

https://api.planet.com/data/v1/stats

In this case, it's a good way to understand how filters work without being overwhelmed with results.  We'll POST a JSON filter definition to the stats endpoint to see how many items match our search criteria.


Using a Search Filter
-------------------------------

Search filters should have the following properties:

  * **Type** (`type`) - The type of filter being used
  * **Configuration** (`config`) - The configuration for this filter
  * **Field Name** (`field_name`) - The field on which to filter

For this example, let's use a filter to get some stats on what data is available for Planet Scope (3 Band) and Rapid Eye (Ortho Tile) products taken from 2013 until now:

In [6]:
stats_url = 'https://api.planet.com/data/v1/stats'

# Specify the sensors/satellites or "item types" to include in our results
item_types = ["PSScene3Band", "REOrthoTile"]

# Create filter object for all imagery captured between 2013-01-01 and present.
date_filter = {
    "type": "DateRangeFilter", # Type of filter -> Date Range
    "field_name": "acquired", # The field to filter on: "acquired" -> Date on which the "image was taken"
    "config": {
        "gte": "2013-01-01T00:00:00.000Z", # "gte" -> Greater than or equal to
    }
}



Depending on the filter type, some requests may need an interval field:

The following intervals are possible:

  * year
  * month
  * week
  * day
  * hour

An interval must be provided with the request so that the number of matching items can be aggregated. We'll use an interval with our date filter.  In this case, we'll get the number of `PSScene3Band` + `REOrthoTile` scenes acquired each year from 2013 to present.

Now let's perform our request using the date_filter filter we created above:


In [9]:
# Construct the request.
request = {
    "item_types" : item_types,
    "interval" : "year",
    "filter" : date_filter
}

# Send the POST request to the API stats endpoint
res = session.post(stats_url, json=request)

# Print response
pprint(res.json())

{'buckets': [{'count': 3094771, 'start_time': '2013-01-01T00:00:00.000000Z'},
             {'count': 3596153, 'start_time': '2014-01-01T00:00:00.000000Z'},
             {'count': 8492329, 'start_time': '2015-01-01T00:00:00.000000Z'},
             {'count': 39665025, 'start_time': '2016-01-01T00:00:00.000000Z'},
             {'count': 210842253, 'start_time': '2017-01-01T00:00:00.000000Z'},
             {'count': 287821006, 'start_time': '2018-01-01T00:00:00.000000Z'},
             {'count': 188101104, 'start_time': '2019-01-01T00:00:00.000000Z'}],
 'interval': 'year',
 'utc_offset': '+0h'}


Good Job! You've received a response from the API that contains some statistics like item counts for the search criteria!

<div class="alert alert-info">

**Exercise:** Create a new date filter to find data from before 2013.

</div>

In [ ]:
# Fill in this filter to complete the exercise! 
date_filter2 = {
}

request = {
    "item_types" : item_types,
    "interval" : "year",
    "filter" : date_filter
}

res = session.post(stats_url, json=request)
p(res.json())

---

Next, let's take a closer look at some of the available filters:

## Filter types

The Planet Data API supports several filter types. The most common are the following:

### Field Filters

* `DateRangeFilter`
* `RangeFilter`
* `StringInFilter`
* `PermissionFilter`
* `GeometryFilter`

### Logical Filters

* `NotFilter`
* `AndFilter`
* `OrFilter`

More information and examples on filter types can be found at the [API Docs](https://www.planet.com/docs/reference/data-api/search-api/). 

---

## (Field) Filter Type examples:

#### `DateRangeFilter`

Find imagery that was `acquired` or `published` before, after or between certain dates.

```
{
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gt": "2016-01-01T00:00:00Z",
    "lte": "2016-03-01T00:00:00Z"
  }
}
```

The upper or lower bound may be omitted.

#### `RangeFilter`

Find imagery that has a metadata that matches a number within a range of numbers.

```
{
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lt": 0.2,
    "gt": 0.1
  }
}
```

The following **operators** are supported by the Data API's `DateRangeFilter` and `RangeFilter`:
* `gt`: Greater Than
* `gte`: Greater Than or Equal To
* `lt`: Less Than
* `lte`: Less Than or Equal To

#### `StringInFilter`

Find imagery that has a metadata that matches a string within the array of provided strings.


```
{
  "type": "StringInFilter",
  "field_name": "instrument",
  "config": ["PS2"]
}
```

#### `PermissionFilter`

Find data which has assets that are accessible by the user.

```
{
  "type": "PermissionFilter",
  "config": ["assets.analytic:download"]
}
```

***Note:*** `assets:download` means *any* downloadable asset.

#### `GeometryFilter`

Find data contained within a given geometry. The filter's config value may be any valid GeoJSON object.

```
{
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -120.27282714843749,
          38.348118547988065
        ],
        [
          -120.27282714843749,
          38.74337300148126
        ],
        [
          -119.761962890625,
          38.74337300148126
        ],
        [
          -119.761962890625,
          38.348118547988065
        ],
        [
          -120.27282714843749,
          38.348118547988065
        ]
      ]
    ]
  }
}
```

A few quick ways to get a GeoJSON geometry to use in your search:

* Draw an Area of Interst (AOI) in [Planet Explorer](http://planet.com/explorer), and use the API button to see the geometry filter config.
* Use your favorite GIS tools like [Quantum GIS (QGIS)](http://www.qgis.org) and export GeoJSON.
* Draw a polygon in [GeoJSON.io](http://geojson.io).

Make sure the `config` property in the geometry filter is in the right format, which should be similar to a `feature.geometry` property in a GeoJSON object. 

---

Let's try a few more requests and get some more stats, this time using different filters:

In [14]:
# Search for imagery only from PlanetScope satellites that have a PS2 telescope (a.k.a "Dove classic")

# Setup item types
item_types = ["PSScene3Band"]

# Setup a filter for instrument type
instrument_filter = {
    "type": "StringInFilter",
    "field_name": "instrument",
    "config": ["PS2"]
}

# Setup the request
request = {
    "item_types" : item_types,
    "interval" : "year",
    "filter" : instrument_filter
}

# Send the POST request to the API stats endpoint
res = session.post(stats_url, json=request)

pprint(res.json())

{'buckets': [{'count': 1540419, 'start_time': '2015-01-01T00:00:00.000000Z'},
             {'count': 32228791, 'start_time': '2016-01-01T00:00:00.000000Z'},
             {'count': 206744916, 'start_time': '2017-01-01T00:00:00.000000Z'},
             {'count': 284909476, 'start_time': '2018-01-01T00:00:00.000000Z'},
             {'count': 176522299, 'start_time': '2019-01-01T00:00:00.000000Z'}],
 'interval': 'year',
 'utc_offset': '+0h'}


<div class="alert alert-info">

**Exercise:** Create a new filter that finds all data from PS0 or PS1 telescopes.

</div>